In [295]:
%reset -f

In [296]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RepeatedKFold
from math import sqrt
from scipy import stats
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, learning_curve
# import ifsg
import json
import re
import glob, os #I ADDED
from sklearn.svm import SVR, LinearSVR #I ADDED
from sklearn.tree import DecisionTreeRegressor #I ADDED

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

In [297]:
file = "W:\\Work\\Assense\\ML\\DataFrames\\NTT_cog_31_03_19.json"
with open(file) as train_file:
    dict_train = json.load(train_file)
sol=[]
no_r=[]
df = pd.DataFrame.from_dict(dict_train)
print(df.shape)

for col in list(df.columns):
    if re.search('.grade_no_round', col):
        sol.append(col)
    elif not re.search('.grade_no_round', col) and not re.search('.min_max', col) and not re.search('cog', col):
        no_r.append(col)

df_gnr= df[sol]
df_features= df[no_r]

#print(no_r)
#df_features
#print(df_gnr.columns)
# print(df_features.columns)
# print(df.columns)



(2, 31)


**Features**

In [298]:
df_features=df[['ang.r_t1stSolution_assNoQ45','ang.r_t1stGrab_assNoQ45', 
                'pzl.r_nSwappedPieces_ass', 'pzl.r_at1stSwap_ass', 'pzl.r_atSwap_ass', 'pzl.r_fSwapsVsMinSwaps_ass',
                'shp.r_t1stGrab_ass', 'shp.r_sAccuracy_ws', 'pzl.r_sAccuracy_ws', 'ang.s_sAccuracy_wsNoQ45',
                'ang.r_tCompletion_assNoQ45','pzl.r_tCompletion_ass','shp.r_tCompletion_ass',
                'ang.s_sAccuracy_sumNoQ45', 'pzl.r_sAccuracy_sum','shp.r_sAccuracy_sum']]
df_features.round(1).head()

,ang.r_t1stSolution_assNoQ45,ang.r_t1stGrab_assNoQ45,pzl.r_nSwappedPieces_ass,pzl.r_at1stSwap_ass,pzl.r_atSwap_ass,pzl.r_fSwapsVsMinSwaps_ass,shp.r_t1stGrab_ass,shp.r_sAccuracy_ws,pzl.r_sAccuracy_ws,ang.s_sAccuracy_wsNoQ45,ang.r_tCompletion_assNoQ45,pzl.r_tCompletion_ass,shp.r_tCompletion_ass,ang.s_sAccuracy_sumNoQ45,pzl.r_sAccuracy_sum,shp.r_sAccuracy_sum
5C9D58E657685F4BE56FB09B,-0.3,-0.3,-0.2,-0.3,-0.4,-1.1,-0.3,2.1,3.8,2.8,0.2,-0.3,0.8,12.0,24,6.0
5C9D58FB57685F4BE56FB09C,-0.1,-0.1,0.2,-0.7,-0.8,-0.8,-0.6,4.7,1.1,1.8,0.1,-0.2,0.0,11.0,21,9.0


**Cognetive Models**

In [299]:
df_cog=df[['cog_cognitive_ability_total_score_w_shp_220119.grade_no_round','cog_spatial_reasoning_w_shp_220119.grade_no_round',
           'cog_performance_IQ_w_shp_220119.grade_no_round', 'cog_verbal_reasoning_150119.grade_no_round',
           'cog_prossecing_speed_wAcc_w_shp_280119.grade_no_round' ]]

df_cog.round(1).head()

,cog_cognitive_ability_total_score_w_shp_220119.grade_no_round,cog_spatial_reasoning_w_shp_220119.grade_no_round,cog_performance_IQ_w_shp_220119.grade_no_round,cog_verbal_reasoning_150119.grade_no_round,cog_prossecing_speed_wAcc_w_shp_280119.grade_no_round
5C9D58E657685F4BE56FB09B,5.1,5.1,5.4,4.9,5.3
5C9D58FB57685F4BE56FB09C,4.9,4.8,5.1,4.6,5.4


In [300]:
file_wm = "W:\\Work\\Assense\\ML\\DataFrames\\NTT_wm_31_03_19.json"
with open(file_wm) as train_file:
    dict_train = json.load(train_file)
sol1=[]
no_r1=[]
df_wm = pd.DataFrame.from_dict(dict_train)
print(df_wm.shape)

for col in list(df_wm.columns):
    if re.search('.grade_no_round', col):
        sol1.append(col)
    elif not re.search('.grade_no_round', col) and not re.search('.min_max', col) and not re.search('cog', col):
        no_r1.append(col)
print(no_r1)
df_gnr1= df_wm[sol1]
df_features1= df_wm[no_r1]

# print(df_gnr1.columns)
# print(df_features1.columns)
# print(df_wm.columns)

(2, 39)
['shp.r_sAccuracy_ws', 'pzl.r_sAccuracy_ws', 'pzl.r_fSwapsVsMinSwaps_ass', 'pzl.r_at1stSwap_ass', 'ang.r_t1stGrab_assNoQ45', 'ang.s_sAccuracy_wsNoQ45', 'ang.r_nObjectSelect_assNoQ45', 'ang.r_nSolutions_assNoQ45', 'shp.r_nSelectedShp_ass', 'shp.r_t1stGrab_ass', 'pzl.r_tCompletion_ass', 'pzl.C#.t_nSwitchings', 'ang.r_tCompletion_assNoQ45', 'shp.r_tCompletion_ass', 'pzl.C#.t_tCompletionToDone', 'ang.r_tCompletionToDone_assNoQ45', 'shp.r_tCompletionToDone_ass', 'ang.r_tDone_assNoQ45', 'shp.r_tDone_ass', 'pzl.r_nSwappedPieces_ass', 'shp.r_sAccuracy_sum', 'pzl.r_sAccuracy_sum', 'pzl.r_atSwap_ass', 'ang.s_sAccuracy_sumNoQ45', 'wm_Planning_w_shp_230119', 'wm_Decisiveness_230119', 'wm_Self_Monitoring_200219', 'wm_Thoroughness_no_shp_210119', 'wm_Time_Management_w_shp_170119']


**Work Methodolegies**

In [301]:
df_wm=df_wm[['wm_Planning_w_shp_230119.grade_no_round','wm_Decisiveness_230119.grade_no_round',
          'wm_Thoroughness_no_shp_210119.grade_no_round', 'wm_Time_Management_w_shp_170119.grade_no_round'
            ,'wm_Self_Monitoring_200219.grade_no_round']]
df_wm.round(1)

,wm_Planning_w_shp_230119.grade_no_round,wm_Decisiveness_230119.grade_no_round,wm_Thoroughness_no_shp_210119.grade_no_round,wm_Time_Management_w_shp_170119.grade_no_round,wm_Self_Monitoring_200219.grade_no_round
5C9D58E657685F4BE56FB09B,4.9,4.5,5.7,5.1,4.3
5C9D58FB57685F4BE56FB09C,4.5,4.6,4.0,5.2,3.8


In [302]:
df_tot = df_features.join(df_cog)
df_tot = df_tot.join(df_wm)
df_tot.round(1)

,ang.r_t1stSolution_assNoQ45,ang.r_t1stGrab_assNoQ45,pzl.r_nSwappedPieces_ass,pzl.r_at1stSwap_ass,pzl.r_atSwap_ass,pzl.r_fSwapsVsMinSwaps_ass,shp.r_t1stGrab_ass,shp.r_sAccuracy_ws,pzl.r_sAccuracy_ws,ang.s_sAccuracy_wsNoQ45,ang.r_tCompletion_assNoQ45,pzl.r_tCompletion_ass,shp.r_tCompletion_ass,ang.s_sAccuracy_sumNoQ45,pzl.r_sAccuracy_sum,shp.r_sAccuracy_sum,cog_cognitive_ability_total_score_w_shp_220119.grade_no_round,cog_spatial_reasoning_w_shp_220119.grade_no_round,cog_performance_IQ_w_shp_220119.grade_no_round,cog_verbal_reasoning_150119.grade_no_round,cog_prossecing_speed_wAcc_w_shp_280119.grade_no_round,wm_Planning_w_shp_230119.grade_no_round,wm_Decisiveness_230119.grade_no_round,wm_Thoroughness_no_shp_210119.grade_no_round,wm_Time_Management_w_shp_170119.grade_no_round,wm_Self_Monitoring_200219.grade_no_round
5C9D58E657685F4BE56FB09B,-0.3,-0.3,-0.2,-0.3,-0.4,-1.1,-0.3,2.1,3.8,2.8,0.2,-0.3,0.8,12.0,24,6.0,5.1,5.1,5.4,4.9,5.3,4.9,4.5,5.7,5.1,4.3
5C9D58FB57685F4BE56FB09C,-0.1,-0.1,0.2,-0.7,-0.8,-0.8,-0.6,4.7,1.1,1.8,0.1,-0.2,0.0,11.0,21,9.0,4.9,4.8,5.1,4.6,5.4,4.5,4.6,4.0,5.2,3.8


In [304]:
file = "W:\\Work\\Assense\\ML\\DataFrames\\NTT_sjt_31_03_19.json"
sjt1=[]
with open(file) as train_file:
    dict_train = json.load(train_file)
df_sjt = pd.DataFrame.from_dict(dict_train)
for col in list(df_sjt.columns):
    if re.search('sjt.', col):
        sjt1.append(col)
print(df.shape)
# df_sjt
df_sjt[sjt1]


(2, 31)


,sjt.SJTRoundUserAnswers.7,sjt.SJTRoundUserQuestions.10,sjt.SJTRoundUserQuestions.21,sjt.SJTRoundUserAnswers.14,sjt.SJTRoundUserAnswers.21,sjt.SJTRoundUserAnswersString.19,sjt.SJTRoundUserAnswers.17,sjt.SJTRoundUserAnswers.6,sjt.SJTRoundUserAnswers.8,sjt.SJTRoundUserAnswersString.20,sjt.SJTRoundUserAnswersString.5,sjt.SJTRoundUserAnswersString.15,sjt.SJTRoundUserAnswers.1,sjt.SJTRoundUserAnswersString.18,sjt.SJTRoundUserAnswersString.14,sjt.SJTRoundUserAnswers.15,sjt.C#.Empathy,sjt.SJTRoundUserAnswers_teamwork_jap_scaled,sjt.SJTRoundUserAnswers_eq_jap,sjt.SJTRoundUserAnswers_eq_heb_scaled,sjt.SJTRoundUserAnswersString.4,sjt.SJTRoundUserAnswers.9,sjt.SJTRoundUserQuestions.7,sjt.SJTRoundUserQuestions.18,sjt.C#.Collegial Behavior,sjt.SJTRoundUserAnswers_empathy_jap_scaled,sjt.SJTRoundUserAnswersString.1,sjt.SJTRoundUserQuestions.6,sjt.SJTRoundUserAnswers.0,sjt.SJTRoundUserAnswers_consienciousness_heb,sjt.SJTRoundUserQuestions.1,sjt.SJTRoundUserAnswersString.13,sjt.SJTRoundUserAnswers.22,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap,sjt.SJTRoundUserQuestions.20,sjt.SJTRoundUserAnswers_teamwork_heb_scaled,sjt.SJTRoundUserQuestions.2,sjt.SJTRoundUserQuestions.12,sjt.SJTRoundUserAnswers.18,sjt.SJTRoundUserQuestions.0,sjt.C#.EQ,sjt.SJTRoundUserQuestions.4,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap_scaled,sjt.SJTRoundUserAnswers.12,sjt.SJTRoundUserAnswers.4,sjt.SJTRoundUserAnswers.5,sjt.SJTRoundUserQuestions.3,sjt.SJTRoundUserAnswers.11,sjt.SJTRoundUserAnswers.13,sjt.SJTRoundUserQuestions.8,sjt.C#.Empathy_scaled,sjt.SJTRoundUserAnswers_empathy_heb,sjt.SJTRoundUserQuestions.19,sjt.SJTRoundUserAnswers.2,sjt.SJTRoundUserAnswersString.8,sjt.SJTRoundUserAnswersString.3,sjt.SJTRoundUserQuestions.16,sjt.SJTRoundUserAnswersString.0,sjt.SJTRoundUserAnswersString.17,sjt.SJTRoundUserAnswers_teamwork_heb,sjt.SJTRoundUserQuestions.14,sjt.SJTRoundUserAnswers_collegial_behavior_jap_scaled,sjt.SJTRoundUserAnswers.3,sjt.SJTRoundUserAnswers_knowledge_of_norms_heb,sjt.SJTRoundUserQuestions.15,sjt.C#.Conscientiousness_scaled,sjt.SJTRoundUserAnswers_knowledge_of_norms_heb_scaled,sjt.SJTRoundUserAnswers_collegial_behavior_heb_scaled,sjt.SJTRoundUserQuestions.9,sjt.C#.Knowledge Of Social Organizational Norms,sjt.C#.Collegial Behavior_scaled,sjt.SJTRoundUserAnswersString.2,sjt.SJTRoundUserAnswers_consienciousness_jap_scaled,sjt.SJTRoundUserAnswers.20,sjt.SJTRoundUserAnswers_consienciousness_heb_scaled,sjt.SJTRoundUserAnswers.19,sjt.SJTRoundUserAnswersString.12,sjt.SJTRoundUserAnswers_teamwork_jap,sjt.SJTRoundUserAnswers_empathy_jap,sjt.SJTRoundUserAnswers_eq_jap_scaled,sjt.SJTRoundUserAnswers_collegial_behavior_jap,sjt.SJTRoundUserAnswersString.11,sjt.SJTRoundUserAnswers_eq_heb,sjt.SJTRoundUserQuestions.22,sjt.SJTRoundUserAnswers_empathy_heb_scaled,sjt.SJTRoundUserAnswersString.10,sjt.C#.Knowledge Of Social Organizational Norms_scaled,sjt.SJTRoundUserAnswers.10,sjt.SJTRoundUserAnswersString.16,sjt.SJTRoundUserAnswersString.6,sjt.SJTRoundUserQuestions.13,sjt.SJTRoundUserAnswers_consienciousness_jap,sjt.SJTRoundUserAnswers_collegial_behavior_heb,sjt.C#.Conscientiousness,sjt.SJTRoundUserQuestions.17,sjt.C#.EQ_scaled,sjt.SJTRoundUserAnswersString.9,sjt.SJTRoundUserQuestions.11,sjt.SJTRoundUserAnswersString.7,sjt.SJTRoundUserAnswersString.21,sjt.SJTRoundUserAnswersString.22,sjt.SJTRoundUserQuestions.5,sjt.SJTRoundUserAnswers.16
5C9D58FB57685F4BE56FB09C,2.0,矢沢亜美についてどう思いますか？,\n 通常、チームワークにおいて最も重要\n なことはあなたにとって...,2.0,1.0,チームをもっと注意深く監督していれば、\nこのような状況は避けられたと思う。,1.0,4.0,1.0,チームメンバーが各自責任を持ちつつ、メンバ\nー間の連絡が可能な環境。,稲山聡に謝り、正しいデータが出るま\nで会議を延期するよう要求する。,稲山聡は山田隆を尊敬しない。,2.0,チーム全員が等しくこのミスの責任者だと思う。,緊張感のある仕事上の関係。,2.0,3.5,None,6.0,None,厳格,3.0,1から6のスケールの中で榎原たかる\n の矢沢亜美への要求を あなたはどう評価しますか？,このミスは結局だれの責任\n だとあなたは思いますか？,4.4375,None,稲山聡を介入して問題の解決を考えてもらう。,この状況で発生したミスについてあな\n たはどう思いますか？,2.0,3.954545,あなたが矢沢亜美だったらどう対応しますか？,自分の立場を犠牲にしてでも彼女が早く\n出れるように上司に説得する。,1.0,4.783784,\n あなた自身を知るかぎり、\n どのような環境でチームの一員として働くことを望み...,None,あなたが矢沢亜美だったら榎原たかる\n のことをどう思いますか？,山田隆が姉の結婚式より仕事を優先し\n てほしいと矢沢亜美に要求する状況を..

$****Accuracy****$

In [305]:
#df_sjt['a2d.r_sAccuracy_sumNoQ12']/15*5+1
7/17*5+1

3.0588235294117645

In [306]:
df_sjt_jap = df_sjt[[
    'sjt.SJTRoundUserAnswers_teamwork_jap',
    'sjt.SJTRoundUserAnswers_eq_jap',
    'sjt.SJTRoundUserAnswers_empathy_jap','sjt.SJTRoundUserAnswers_knowledge_of_norms_jap',
    'sjt.SJTRoundUserAnswers_consienciousness_jap','sjt.SJTRoundUserAnswers_collegial_behavior_jap'
    ,'a2d.r_sAccuracy_sumNoQ12'
    ]]
df_sjt_jap.round(1)

,sjt.SJTRoundUserAnswers_teamwork_jap,sjt.SJTRoundUserAnswers_eq_jap,sjt.SJTRoundUserAnswers_empathy_jap,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap,sjt.SJTRoundUserAnswers_consienciousness_jap,sjt.SJTRoundUserAnswers_collegial_behavior_jap,a2d.r_sAccuracy_sumNoQ12
5C9D58FB57685F4BE56FB09C,3.9,6.0,3.5,4.8,3.9,4.4,12.0


*Appending sjt columns(includs NaN case)*

In [307]:
df_jstJAP_a2d = pd.DataFrame(columns = ['sjt.SJTRoundUserAnswers_teamwork_jap','sjt.SJTRoundUserAnswers_eq_jap',
                                        'sjt.SJTRoundUserAnswers_empathy_jap','sjt.SJTRoundUserAnswers_knowledge_of_norms_jap',
                                        'sjt.SJTRoundUserAnswers_consienciousness_jap','sjt.SJTRoundUserAnswers_collegial_behavior_jap',
                                       'a2d.r_sAccuracy_sumNoQ12'])

df_jstJAP_a2d = df_jstJAP_a2d.append(df_sjt_jap, sort=True)
df_jstJAP_a2d = df_jstJAP_a2d[['sjt.SJTRoundUserAnswers_teamwork_jap','sjt.SJTRoundUserAnswers_eq_jap',
                                        'sjt.SJTRoundUserAnswers_empathy_jap','sjt.SJTRoundUserAnswers_knowledge_of_norms_jap',
                                        'sjt.SJTRoundUserAnswers_consienciousness_jap','sjt.SJTRoundUserAnswers_collegial_behavior_jap',
                              'a2d.r_sAccuracy_sumNoQ12']]
df_jstJAP_a2d.round(1)


,sjt.SJTRoundUserAnswers_teamwork_jap,sjt.SJTRoundUserAnswers_eq_jap,sjt.SJTRoundUserAnswers_empathy_jap,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap,sjt.SJTRoundUserAnswers_consienciousness_jap,sjt.SJTRoundUserAnswers_collegial_behavior_jap,a2d.r_sAccuracy_sumNoQ12
5C9D58FB57685F4BE56FB09C,3.9,6.0,3.5,4.8,3.9,4.4,12.0


*Ari2d*

In [308]:
df_jstJAP_a2d['a2d.r_sAccuracy_sumNoQ12'] = df_sjt['a2d.r_sAccuracy_sumNoQ12']/15*5+1
df_jstJAP_a2d

,sjt.SJTRoundUserAnswers_teamwork_jap,sjt.SJTRoundUserAnswers_eq_jap,sjt.SJTRoundUserAnswers_empathy_jap,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap,sjt.SJTRoundUserAnswers_consienciousness_jap,sjt.SJTRoundUserAnswers_collegial_behavior_jap,a2d.r_sAccuracy_sumNoQ12
5C9D58FB57685F4BE56FB09C,3.916667,6.0,3.5,4.783784,3.916667,4.4375,5.0


In [309]:
for i in range(0, df_jstJAP_a2d.shape[0]):
    for j in range(0, df_jstJAP_a2d.shape[1]):
        df_jstJAP_a2d.iloc[i,j]=df_jstJAP_a2d.iloc[i,j]+np.random.uniform(-0.3,0.3)
        if df_jstJAP_a2d.iloc[i,j]>6:
            df_jstJAP_a2d.iloc[i,j]=6
        if df_jstJAP_a2d.iloc[i,j]<1:
            df_jstJAP_a2d.iloc[i,j]=1
df_jstJAP_a2d.round(1)

,sjt.SJTRoundUserAnswers_teamwork_jap,sjt.SJTRoundUserAnswers_eq_jap,sjt.SJTRoundUserAnswers_empathy_jap,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap,sjt.SJTRoundUserAnswers_consienciousness_jap,sjt.SJTRoundUserAnswers_collegial_behavior_jap,a2d.r_sAccuracy_sumNoQ12
5C9D58FB57685F4BE56FB09C,3.9,5.9,3.4,4.5,4.2,4.1,4.8


Joining sjt to final df

In [310]:
df_tot =df_tot.join(df_jstJAP_a2d)
df_tot.round(1)


,ang.r_t1stSolution_assNoQ45,ang.r_t1stGrab_assNoQ45,pzl.r_nSwappedPieces_ass,pzl.r_at1stSwap_ass,pzl.r_atSwap_ass,pzl.r_fSwapsVsMinSwaps_ass,shp.r_t1stGrab_ass,shp.r_sAccuracy_ws,pzl.r_sAccuracy_ws,ang.s_sAccuracy_wsNoQ45,ang.r_tCompletion_assNoQ45,pzl.r_tCompletion_ass,shp.r_tCompletion_ass,ang.s_sAccuracy_sumNoQ45,pzl.r_sAccuracy_sum,shp.r_sAccuracy_sum,cog_cognitive_ability_total_score_w_shp_220119.grade_no_round,cog_spatial_reasoning_w_shp_220119.grade_no_round,cog_performance_IQ_w_shp_220119.grade_no_round,cog_verbal_reasoning_150119.grade_no_round,cog_prossecing_speed_wAcc_w_shp_280119.grade_no_round,wm_Planning_w_shp_230119.grade_no_round,wm_Decisiveness_230119.grade_no_round,wm_Thoroughness_no_shp_210119.grade_no_round,wm_Time_Management_w_shp_170119.grade_no_round,wm_Self_Monitoring_200219.grade_no_round,sjt.SJTRoundUserAnswers_teamwork_jap,sjt.SJTRoundUserAnswers_eq_jap,sjt.SJTRoundUserAnswers_empathy_jap,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap,sjt.SJTRoundUserAnswers_consienciousness_jap,sjt.SJTRoundUserAnswers_collegial_behavior_jap,a2d.r_sAccuracy_sumNoQ12
5C9D58E657685F4BE56FB09B,-0.3,-0.3,-0.2,-0.3,-0.4,-1.1,-0.3,2.1,3.8,2.8,0.2,-0.3,0.8,12.0,24,6.0,5.1,5.1,5.4,4.9,5.3,4.9,4.5,5.7,5.1,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5C9D58FB57685F4BE56FB09C,-0.1,-0.1,0.2,-0.7,-0.8,-0.8,-0.6,4.7,1.1,1.8,0.1,-0.2,0.0,11.0,21,9.0,4.9,4.8,5.1,4.6,5.4,4.5,4.6,4.0,5.2,3.8,3.9,5.9,3.4,4.5,4.2,4.1,4.8


In [292]:
file_pcs = "W:\\Work\\Assense\\ML\\DataFrames\\NTT_pcs_31_03_19.json"
with open(file_pcs) as train_file:
    dict_train = json.load(train_file)
sol_pcs=[]
df_pcs = pd.DataFrame.from_dict(dict_train)
print(df_pcs.shape)

for col in list(df_pcs.columns):
    if re.search('.grade_no_round', col):
        sol_pcs.append(col)

df_pcs_gnr= df_pcs[sol_pcs]
df_pcs_gnr.head()


(2, 17)


,wm_Risk_Taking_ShortPCS_210219.grade_no_round,wm_Goal_Orientation_ShortPCS_210219.grade_no_round,wm_Methodicalness_shortPCS_210219.grade_no_round
5C9D58E657685F4BE56FB09B,NaN,NaN,NaN
5C9D58FB57685F4BE56FB09C,2.810846,5.632682,5.779367


In [293]:
df_tot =df_tot.join(df_pcs_gnr)
df_tot.round(1)

,ang.r_t1stSolution_assNoQ45,ang.r_t1stGrab_assNoQ45,pzl.r_nSwappedPieces_ass,pzl.r_at1stSwap_ass,pzl.r_atSwap_ass,pzl.r_fSwapsVsMinSwaps_ass,shp.r_t1stGrab_ass,shp.r_sAccuracy_ws,pzl.r_sAccuracy_ws,ang.s_sAccuracy_wsNoQ45,ang.r_tCompletion_assNoQ45,pzl.r_tCompletion_ass,shp.r_tCompletion_ass,ang.s_sAccuracy_sumNoQ45,pzl.r_sAccuracy_sum,shp.r_sAccuracy_sum,cog_cognitive_ability_total_score_w_shp_220119.grade_no_round,cog_spatial_reasoning_w_shp_220119.grade_no_round,cog_performance_IQ_w_shp_220119.grade_no_round,cog_verbal_reasoning_150119.grade_no_round,cog_prossecing_speed_wAcc_w_shp_280119.grade_no_round,wm_Planning_w_shp_230119.grade_no_round,wm_Decisiveness_230119.grade_no_round,wm_Thoroughness_no_shp_210119.grade_no_round,wm_Time_Management_w_shp_170119.grade_no_round,wm_Self_Monitoring_200219.grade_no_round,sjt.SJTRoundUserAnswers_teamwork_jap,sjt.SJTRoundUserAnswers_eq_jap,sjt.SJTRoundUserAnswers_empathy_jap,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap,sjt.SJTRoundUserAnswers_consienciousness_jap,sjt.SJTRoundUserAnswers_collegial_behavior_jap,a2d.r_sAccuracy_sumNoQ12,wm_Risk_Taking_ShortPCS_210219.grade_no_round,wm_Goal_Orientation_ShortPCS_210219.grade_no_round,wm_Methodicalness_shortPCS_210219.grade_no_round
5C9D58E657685F4BE56FB09B,-0.3,-0.3,-0.2,-0.3,-0.4,-1.1,-0.3,2.1,3.8,2.8,0.2,-0.3,0.8,12.0,24,6.0,5.1,5.1,5.4,4.9,5.3,4.9,4.5,5.7,5.1,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5C9D58FB57685F4BE56FB09C,-0.1,-0.1,0.2,-0.7,-0.8,-0.8,-0.6,4.7,1.1,1.8,0.1,-0.2,0.0,11.0,21,9.0,4.9,4.8,5.1,4.6,5.4,4.5,4.6,4.0,5.2,3.8,NaN,6.0,3.4,4.5,4.0,4.2,4.8,2.8,5.6,5.8


In [294]:
df_tot

,ang.r_t1stSolution_assNoQ45,ang.r_t1stGrab_assNoQ45,pzl.r_nSwappedPieces_ass,pzl.r_at1stSwap_ass,pzl.r_atSwap_ass,pzl.r_fSwapsVsMinSwaps_ass,shp.r_t1stGrab_ass,shp.r_sAccuracy_ws,pzl.r_sAccuracy_ws,ang.s_sAccuracy_wsNoQ45,ang.r_tCompletion_assNoQ45,pzl.r_tCompletion_ass,shp.r_tCompletion_ass,ang.s_sAccuracy_sumNoQ45,pzl.r_sAccuracy_sum,shp.r_sAccuracy_sum,cog_cognitive_ability_total_score_w_shp_220119.grade_no_round,cog_spatial_reasoning_w_shp_220119.grade_no_round,cog_performance_IQ_w_shp_220119.grade_no_round,cog_verbal_reasoning_150119.grade_no_round,cog_prossecing_speed_wAcc_w_shp_280119.grade_no_round,wm_Planning_w_shp_230119.grade_no_round,wm_Decisiveness_230119.grade_no_round,wm_Thoroughness_no_shp_210119.grade_no_round,wm_Time_Management_w_shp_170119.grade_no_round,wm_Self_Monitoring_200219.grade_no_round,sjt.SJTRoundUserAnswers_teamwork_jap,sjt.SJTRoundUserAnswers_eq_jap,sjt.SJTRoundUserAnswers_empathy_jap,sjt.SJTRoundUserAnswers_knowledge_of_norms_jap,sjt.SJTRoundUserAnswers_consienciousness_jap,sjt.SJTRoundUserAnswers_collegial_behavior_jap,a2d.r_sAccuracy_sumNoQ12,wm_Risk_Taking_ShortPCS_210219.grade_no_round,wm_Goal_Orientation_ShortPCS_210219.grade_no_round,wm_Methodicalness_shortPCS_210219.grade_no_round
5C9D58E657685F4BE56FB09B,-0.326197,-0.292597,-0.2306,-0.2934,-0.3900,-1.0575,-0.3473,2.054791,3.826702,2.786262,0.178754,-0.2673,0.7820,12.0,24,6.0,5.084755,5.081388,5.379430,4.854343,5.269741,4.870825,4.475438,5.703170,5.095074,4.270110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5C9D58FB57685F4BE56FB09C,-0.137709,-0.114109,0.2193,-0.7237,-0.7554,-0.8479,-0.6128,4.749447,1.102077,1.796114,0.106798,-0.1835,0.0294,11.0,21,9.0,4.864229,4.837434,5.137168,4.642638,5.352156,4.535809,4.556929,4.047817,5.165706,3.788887,NaN,5.995859,3.361973,4.518929,3.981828,4.228379,4.814754,2.810846,5.632682,5.779367


In [92]:
#Transpose df
df_tot.T

,5C906943152B7F0DDE5683E0,5C9074B1152B7F0DDE5683E1,DT239
ang.r_t1stSolution_assNoQ45,-0.276757,-0.690350,1.180243
ang.r_t1stGrab_assNoQ45,0.059249,-0.593225,0.481593
pzl.r_nSwappedPieces_ass,-0.079600,-0.165400,0.060600
pzl.r_at1stSwap_ass,-0.603000,0.481700,0.291300
pzl.r_atSwap_ass,-0.393200,1.007700,0.912900
pzl.r_fSwapsVsMinSwaps_ass,-0.895400,-0.856500,-0.841400
shp.r_t1stGrab_ass,-0.316900,-0.080400,0.353000
shp.r_sAccuracy_ws,3.000476,-0.571062,3.000476
pzl.r_sAccuracy_ws,2.017256,-2.596855,0.244774
ang.s_sAccuracy_wsNoQ45,1.828782,-4.664097,1.790929


In [228]:
for i in range(0, df.shape[0]):
    for j in range(0, df.shape[1]):
        df.iloc[i,j]=df.iloc[i,j]+np.random.uniform(-0.3,0.3)
        if df.iloc[i,j]>6:
            df.iloc[i,j]=6
        if df.iloc[i,j]<1:
            df.iloc[i,j]=1
df

,ang.r_tCompletion_assNoQ45,pzl.r_tCompletion_ass,shp.r_sAccuracy_sum,pzl.r_sAccuracy_sum,shp.r_tCompletion_ass,ang.s_sAccuracy_sumNoQ45,pzl.r_nSwappedPieces_ass,pzl.r_at1stSwap_ass,shp.r_sAccuracy_ws,ang.r_t1stSolution_assNoQ45,ang.s_sAccuracy_wsNoQ45,pzl.r_sAccuracy_ws,ang.r_t1stGrab_assNoQ45,shp.r_t1stGrab_ass,pzl.r_atSwap_ass,pzl.r_fSwapsVsMinSwaps_ass,cog_cognitive_ability_total_score_w_shp_220119,cog_cognitive_ability_total_score_w_shp_220119.min_max,cog_cognitive_ability_total_score_w_shp_220119.grade_no_round,cog_performance_IQ_w_shp_220119,cog_performance_IQ_w_shp_220119.min_max,cog_performance_IQ_w_shp_220119.grade_no_round,cog_prossecing_speed_wAcc_w_shp_280119,cog_prossecing_speed_wAcc_w_shp_280119.min_max,cog_prossecing_speed_wAcc_w_shp_280119.grade_no_round,cog_spatial_reasoning_w_shp_220119,cog_spatial_reasoning_w_shp_220119.min_max,cog_spatial_reasoning_w_shp_220119.grade_no_round,cog_verbal_reasoning_150119,cog_verbal_reasoning_150119.min_max,cog_verbal_reasoning_150119.grade_no_round
5C8AF6773B08960F8897D115,1.0,1.0,6.0,6.0,1.0,6.0,1.0,1.0,4.655068,1.0,1.0,3.740313,1.0,1.0,1.0,1.0,6.0,6.0,5.085614,6.0,6.0,4.996457,6.0,6.0,4.607588,6.0,6.0,5.179659,6.0,6.0,4.815972


In [79]:
df_jstJAP_a2d.to_excel('df_jstJAP_a2d.xlsx')

In [70]:
#SJT_ISR
df_sjt_isr_a2d = df_sjt[[
    'sjt.SJTRoundUserAnswers_teamwork_heb',
    'sjt.SJTRoundUserAnswers_eq_heb',
    'sjt.SJTRoundUserAnswers_empathy_heb','sjt.SJTRoundUserAnswers_knowledge_of_norms_heb',
   'sjt.SJTRoundUserAnswers_consienciousness_heb','sjt.SJTRoundUserAnswers_collegial_behavior_heb']]

df_sjt_isr_a2d.round(1)



,sjt.SJTRoundUserAnswers_teamwork_heb,sjt.SJTRoundUserAnswers_eq_heb,sjt.SJTRoundUserAnswers_empathy_heb,sjt.SJTRoundUserAnswers_knowledge_of_norms_heb,sjt.SJTRoundUserAnswers_consienciousness_heb,sjt.SJTRoundUserAnswers_collegial_behavior_heb
5C8E1563E0054A3B59405DB6,6.0,4.6,4.1,5.3,5.1,4.9


In [72]:
df_sjt_isr = pd.DataFrame(columns = ['sjt.SJTRoundUserAnswers_teamwork_heb','sjt.SJTRoundUserAnswers_eq_heb',
                                'sjt.SJTRoundUserAnswers_empathy_heb','sjt.SJTRoundUserAnswers_knowledge_of_norms_heb',
                               'sjt.SJTRoundUserAnswers_consienciousness_heb','sjt.SJTRoundUserAnswers_collegial_behavior_heb', 'a2d.r_sAccuracy_sumNoQ12'])

df_sjt_isr_a2d = df_sjt_isr_a2d.append(df_sjt_isr, sort=True)
df_sjt_isr_a2d = df_sjt_isr_a2d[['sjt.SJTRoundUserAnswers_teamwork_heb','sjt.SJTRoundUserAnswers_eq_heb',
                                'sjt.SJTRoundUserAnswers_empathy_heb','sjt.SJTRoundUserAnswers_knowledge_of_norms_heb',
                               'sjt.SJTRoundUserAnswers_consienciousness_heb','sjt.SJTRoundUserAnswers_collegial_behavior_heb', 'a2d.r_sAccuracy_sumNoQ12']]
df_sjt_isr_a2d.round(1)

,sjt.SJTRoundUserAnswers_teamwork_heb,sjt.SJTRoundUserAnswers_eq_heb,sjt.SJTRoundUserAnswers_empathy_heb,sjt.SJTRoundUserAnswers_knowledge_of_norms_heb,sjt.SJTRoundUserAnswers_consienciousness_heb,sjt.SJTRoundUserAnswers_collegial_behavior_heb,a2d.r_sAccuracy_sumNoQ12
5C8E1563E0054A3B59405DB6,6.0,4.6,4.1,5.3,5.1,4.9,NaN


In [59]:
#additional col for Ari2D ISR
df_sjt_isr_a2d['a2d.r_sAccuracy_sumNoQ12']=df['a2d.r_sAccuracy_sumNoQ12']/15*5+1
df_sjt_isr_a2d

C:\NewProg\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,sjt.SJTRoundUserAnswers_teamwork_heb,sjt.SJTRoundUserAnswers_eq_heb,sjt.SJTRoundUserAnswers_collegial_behavior_heb,sjt.SJTRoundUserAnswers_knowledge_of_norms_heb,sjt.SJTRoundUserAnswers_empathy_heb,sjt.SJTRoundUserAnswers_consienciousness_heb,a2d.r_sAccuracy_sumNoQ12
5C861145E0054A3B59405DB3,4.75,4.75,3.857143,4.965517,4.125,5.090909,1.000000
5C86320BE0054A3B59405DB4,6.00,4.75,6.000000,4.103448,4.125,5.090909,4.333333


In [73]:
for i in range(0, df_sjt_isr_a2d.shape[0]):
    for j in range(0, df_sjt_isr_a2d.shape[1]):
        df_sjt_isr_a2d.iloc[i,j]=df_sjt_isr_a2d.iloc[i,j]+np.random.uniform(-0.3,0.3)
        if df_sjt_isr_a2d.iloc[i,j]>6:
            df_sjt_isr_a2d.iloc[i,j]=6
        if df_sjt_isr_a2d.iloc[i,j]<1:
            df_sjt_isr_a2d.iloc[i,j]=1
df_sjt_isr_a2d

,sjt.SJTRoundUserAnswers_teamwork_heb,sjt.SJTRoundUserAnswers_eq_heb,sjt.SJTRoundUserAnswers_empathy_heb,sjt.SJTRoundUserAnswers_knowledge_of_norms_heb,sjt.SJTRoundUserAnswers_consienciousness_heb,sjt.SJTRoundUserAnswers_collegial_behavior_heb,a2d.r_sAccuracy_sumNoQ12
5C8E1563E0054A3B59405DB6,6.0,4.437484,4.076473,5.112263,5.197222,5.022378,NaN


(8, 17)


,wm_Risk_Taking_ShortPCS_210219.grade_no_round,wm_Goal_Orientation_ShortPCS_210219.grade_no_round,wm_Methodicalness_shortPCS_210219.grade_no_round
5C9191294488854086A773F5,2.899713,3.490852,2.475109
5C91912A4488854086A773F6,2.099725,4.083211,4.959430
5C91D7BC4488854086A773F7,1.500128,4.451577,5.760675
5C91F3AA4488854086A773F8,2.291993,4.849880,5.720022
5C942F094488854086A773F9,3.929101,2.471444,1.960806
